In [1]:
import numpy as np
import time 
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
from jupyterthemes import jtplot

In [15]:
def set_ER(erosion_rates):
    erosion_rates[:,0] = initial_ER_rates
    
    if shift_ER_rate == True:
        erosion_rates[:,1] = initial_ER_rates * ER_shift_factor
    
    return erosion_rates

In [5]:
def C14fxn(dt, depth, conc0, scaling_factor):
       
    C14_P0 = SLHL_C14 * scaling_factor   # production rate atoms / g / yr
    thalf = 5730    # half life 14-C, years
    decay = np.log(2) / thalf
    f_tot = 1.72*10**(-2)

    mu = 2.62/160 # g * cm^-2
    A1 = 170.6 # Granger and Smith (2000)
    A2 = 36.75 # ibid.
    L1 = 788.6 # g cm^-2; Granger and Smith (2000)
    L2 = 2688 # g cm*-2; ibid.
    
    muons = [f_tot * (A1*np.e**((-rock_rho * depth[i])/L1) + A2*np.e**((-2.62*depth[i])/L2)) for i in depth]  
    spallation = [C14_P0 * np.e**(-mu * depth[i]) for i in depth]
    production_tot_14 = [spallation[i] + muons[i] for i in depth]
    
    conc14C = [conc0[i]*np.e**(-decay) + (production_tot_14[i] / (decay + 0) * (1 - np.e**(-(decay + 0) * 1))) for i in depth]

    
    return conc14C

In [6]:
def Be10fxn(dt, depth, conc0, scaling_factor):
    
    A1 = 170.6 # Granger and Smith (2000)
    A2 = 36.75 # ibid.
    L1 = 788.6 # g cm^-2; Granger and Smith (2000)
    L2 = 2688 
    
    Be10_P0 = SLHL_Be10 * scaling_factor
    decay = 4.998e-7    
    B_Be = 0.026 # Granger and Smith 2000
    L3 = 4360 # g cm^-2 Granger and Smith 2000
    f_tot = 5.6*10**(-4)
    
    f_muons_10 = [0.026 * np.e**(-rock_rho * depth[i] / L3) for i in depth]
    neg_muons_10 = [f_tot * (A1*np.e**((-rock_rho * depth[i])/L1) + A2*np.e**((-rock_rho*depth[i])/L2)) for i in depth]
    spallogenic_10 = [Be10_P0 * np.e**(-mu * depth[i]) for i in depth]
    production_tot_10 = [f_muons_10[i] + spallogenic_10[i] + neg_muons_10[i] for i in depth]
    
    conc10Be = [conc0[i] * np.e**(-decay) + (production_tot_10[i] / (decay + 0) * (1 - np.e**(-(decay + 0) * 1))) for i in depth]    

    return conc10Be

In [33]:
def CRN_fxn(conc10Be, conc14C, ER, dat_lin):
    
    conc10Be = Be10fxn(dt, dat_lin, conc10Be, scaling_factor)
    conc14C = C14fxn(dt, dat_lin, conc14C, scaling_factor)    
    
    depths = dat_lin[dat_lin > ER]
    minus = len(dat_lin) - len(depths)

    sec_Befxn = interp1d(x = (depths - ER), y = conc10Be[minus:], kind = 'cubic', fill_value = "extrapolate")
    conc10Be[:-minus] = sec_Befxn(x = dat_lin[:-minus])
    conc10Be[-1] = 0
    
    sec_Cfxn = interp1d(x = (depths - ER), y = conc14C[minus:], kind = 'cubic', fill_value = "extrapolate")
    conc14C[:-minus] = sec_Cfxn(x = dat_lin[:-minus])
    conc14C[-1] = 0
    
    C_Be_ratio = np.divide(conc14C,conc10Be)
    
    return conc10Be, conc14C, C_Be_ratio

In [39]:
def CRN_comp_loop_fxn(total_time, initial_ER):
    dat_lin = np.arange(0,max_depth[1],1)
    
    conc14C = [0] * len(dat_lin)
    conc10Be = [0] * len(dat_lin)
    C_Be_surf = np.empty((total_time,))
    C_surf = np.empty((total_time,))
    Be_surf = np.empty((total_time,))
    C_tot = np.empty((len(dat_lin),total_time))
    Be_tot = np.empty((len(dat_lin),total_time))
            
    ER = initial_ER

    for i in range(0, total_time):
        conc10Be, conc14C, C_Be_ratio = CRN_fxn(conc10Be, conc14C, ER, dat_lin)  
        C_tot[:,i] = conc14C
        Be_tot[:,i] = conc10Be
        C_Be_surf[i,] = np.average(C_Be_ratio[0:6])
        C_surf[i,] = np.average(conc14C[0:6])
        Be_surf[i,] = np.average(conc10Be[0:6])
            
    ratios = [(C_tot[0,i] / Be_tot[0,i]) for i in range(total_time)]

    export_mat = np.empty((total_time,4))
    export_mat[:,0] = C_surf
    export_mat[:,1] = Be_surf
    export_mat[:,2] = ratios
    export_mat[0,3] = ER

    C_export = C_tot
    Be_export = Be_tot

    print('Comparison Be10 / C14 loop complete.')
    
    np.savetxt('CON_C14_10Be_ER' + str(ER) + '_expmat.csv',
               export_mat,
               delimiter = ',')

    return C_tot, Be_tot, C_Be_surf, C_surf, Be_surf, ratios